### 게시판 10개 PAGE 크롤링   

driver.get('https://www.durunubi.kr/4-2-0-walk-mobility.do?curPage=91&rgn_idx=&dist_num=&distance=&time=&difficulty=')에서 
'Page=11&rgn'에 앞자리 숫자를 1~9까지로 변경하면서 크롤링.    

In [ ]:
import os
import re
from urllib.error import HTTPError
import urllib.request 
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.alert import Alert
from selenium import webdriver as wd
import pandas as pd

driver = wd.Chrome(executable_path = './chromedriver')
#driver.get('https://www.durunubi.kr/4-2-0-walk-mobility.do')
#driver.get('https://www.durunubi.kr/4-2-0-walk-mobility.do?curPage=11&rgn_idx=&dist_num=&distance=&time=&difficulty=')
driver.get('https://www.durunubi.kr/4-2-0-walk-mobility.do?curPage=91&rgn_idx=&dist_num=&distance=&time=&difficulty=')

## 사이트접속

for page in range(3, 13):
    driver.find_element_by_xpath('//*[@id="content"]/div/nav/ul/li[%s]/a' %page).click()
    for detail in range(1, 21):
        #길 상세페이지 열기
        driver.find_element_by_xpath('//*[@id="frm"]/div[%s]/a/div[2]/div[1]' %detail).click()
        #driver.implicitly_wait(5)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        #세부에 필요한 정보만 가져오기
        road_name = soup.select('div.container > h1')
        road_address = soup.select('div.load-info > strong')
        #주행거리
        road_distance = soup.select('div.container > div.load-info > span.km.ml-20')
        #주행시간
        road_time = soup.select('div.container > div.load-info > span.time.ml-20')
        #난이도
        road_level = soup.select('div.col-lg-4 > table > tbody > tr > td:nth-child(1)')
        #별점
        road_star = soup.select('div.col-lg-4 > table > tbody > tr > td:nth-child(2)')
        #순환형
        road_type = soup.select('div.col-lg-4 > table > tbody > tr > td:nth-child(3)')
        #코스개요
        road_outline = soup.select('div.col-lg-8 > div:nth-child(2)')
        #관광포인트
        road_point = soup.select('div.col-lg-8 > div:nth-child(4)')
        #여행자정보
        road_info = soup.select('div.col-lg-8 > div:nth-child(6)')
        
        
        # 전체정보합치기
        all_info = road_name + road_address + road_distance + road_time + road_level + road_star + road_type + road_outline + road_point + road_info 
        road_summary = []
            
        for i in all_info:
            j = i.text
            road_summary.append(j)
        
        for title in road_name:
            road_title = title.text        
       
        # 띄어쓰기 제거
        road_summary = [line.rstrip() for line in road_summary]
        road_summary = [line.lstrip() for line in road_summary]
        
        #데이터프레임 만들기, csv저장하기
        df = pd.DataFrame(road_summary, index=['코스이름', '주소', '거리', '시간', '난이도', '평점', '코스형태', 'none', 'none', '코스개요', 'none', '코스포인트', 'none', '여행자정보'])
        df.drop(['none'], inplace=True)        
        
        df.to_csv('{}.csv'.format(road_title), encoding="euc_kr")
         # encoding 다른버전 'utf-8-sig' csv 파일을 그대로. 

        #이미지 링크 기져오기
        link = []
        for i in range(1, 2): 
            try:
                img =driver.find_element_by_xpath('//*[@id="swiper_images_01"]/div/div[1]/div[1]/div/a/img')     
            except : 
                pass
            link.append(img.get_attribute('src'))
            #print(link)
        
        # 폴더에 들어갈 타이틀 작성
        for title in road_name:
            
            keyword = title.text
        
        #새로운 폴더생성
#        if not os.path.isdir('./{}'.format(keyword)):
#            os.mkdir('./{}'.format(keyword))
            
        
        count = 0
        for url in link:
            # 링크가 두 개 이상일시
            # count += 1
            # urllib.request.urlretrieve(url, './{}'.format(keyword)+str(count)+'.jpg')
            urllib.request.urlretrieve(url, './{}'.format(keyword) + '.jpg')

            driver.back() 



### 전체 원본 크롤링 소스

In [ ]:
import os
import re
from urllib.error import HTTPError
import urllib.request 
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.alert import Alert
from selenium import webdriver as wd
import pandas as pd

driver = wd.Chrome(executable_path = './chromedriver')
#driver.get('https://www.durunubi.kr/4-2-0-walk-mobility.do')
#driver.get('https://www.durunubi.kr/4-2-0-walk-mobility.do?curPage=11&rgn_idx=&dist_num=&distance=&time=&difficulty=')
#driver.get('https://www.durunubi.kr/4-2-0-walk-mobility.do?curPage=21&rgn_idx=&dist_num=&distance=&time=&difficulty=')

## 사이트접속
for board in range(3, 10):
    driver.get('https://www.durunubi.kr/4-2-0-walk-mobility.do?curPage=%s1&rgn_idx=&dist_num=&distance=&time=&difficulty=' %board)
    for page in range(3, 13):
        driver.find_element_by_xpath('//*[@id="content"]/div/nav/ul/li[%s]/a' %page).click()
        for detail in range(1, 21):
            #길 상세페이지 열기
            driver.find_element_by_xpath('//*[@id="frm"]/div[%s]/a/div[2]/div[1]' %detail).click()
            #driver.implicitly_wait(5)
        
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
        
            #세부에 필요한 정보만 가져오기
            road_name = soup.select('div.container > h1')
            road_address = soup.select('div.load-info > strong')
            #주행거리
            road_distance = soup.select('div.container > div.load-info > span.km.ml-20')
            #주행시간
            road_time = soup.select('div.container > div.load-info > span.time.ml-20')
            #난이도
            road_level = soup.select('div.col-lg-4 > table > tbody > tr > td:nth-child(1)')
            #평점
            road_star = soup.select('div.col-lg-4 > table > thead > tr > th:nth-child(2)')
            #순환형
            road_type = soup.select('div.col-lg-4 > table > tbody > tr > td:nth-child(3)')
            #코스개요
            road_outline = soup.select('div.col-lg-8 > div:nth-child(2)')
            #관광포인트
            road_point = soup.select('div.col-lg-8 > div:nth-child(4)')
            #여행자정보
            road_info = soup.select('div.col-lg-8 > div:nth-child(6)')
        
        
            # 전체정보합치기
            all_info = road_name + road_address + road_distance + road_time + road_level + road_star + road_type + road_outline + road_point + road_info 
            road_summary = []
            
            for i in all_info:
                j = i.text
                road_summary.append(j)
        
            for title in road_name:
                road_title = title.text        
       
            # 띄어쓰기 제거
            road_summary = [line.rstrip() for line in road_summary]
            road_summary = [line.lstrip() for line in road_summary]
        
            #데이터프레임 만들기, csv저장하기
            df = pd.DataFrame(road_summary, index=['코스이름', '주소', '거리', '시간', '난이도', '평점', '코스형태', 'none', 'none', '코스개요', 'none', '코스포인트', 'none', '여행자정보'])
            df.drop(['none'], inplace=True)        
        
            df.to_csv('{}.csv'.format(road_title), encoding="utf-8")
            # df.to_csv('text.csv', encoding="utf-8")
 

            #이미지 링크 기져오기
            link = []
            for i in range(1, 2): 
                try:
                    img =driver.find_element_by_xpath('//*[@id="swiper_images_01"]/div/div[1]/div[1]/div/a/img')     
                except : 
                    pass
                link.append(img.get_attribute('src'))
                #print(link)
        
            # 폴더에 들어갈 타이틀 작성
            for title in road_name:
            
                keyword = title.text
        
            #새로운 폴더생성
            if not os.path.isdir('./{}'.format(keyword)):
                os.mkdir('./{}'.format(keyword))
            
        
            count = 0
            for url in link:
            # 링크가 두 개 이상일시
            # count += 1
            # urllib.request.urlretrieve(url, './{}'.format(keyword)+str(count)+'.jpg')
                urllib.request.urlretrieve(url, './{}'.format(keyword) + '.jpg')

                driver.back() 

